In [1]:
search_for = 129
start_from = 0
threads = 10
thread = 7
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.11126065254211426
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 423524321
type: [1, 1, 1, 1, 129] 129
cases of this type: 276922881
100000 20.07466372842338
200000 30.972593536758012
300000 30.59897167291611
400000 30.820415274736444
500000 29.80697961249153
600000 31.94817524757146
700000 30.032701951762284
800000 29.588433456226372
900000 30.268974142974745
1000000 28.851211809768877
1100000 12.797254642114583
1200000 26.882398321607834
1300000 29.579404923765672
1400000 30.295123270201064
1500000 29.109884799818538
1600000 30.51541853365671
1700000 31.225965441599843
1800000 29.934002289111934
1900000 30.676569612659797
2000000 30.261118418206312
2100000 30.778702579476697
2200000 8.021237315253847
2300000 34.77024569673949
2400000 38.69186439649044
2500000 36.671486534362444
2600000 38.06305877410785
2700000 37.56440566580534
2800000 36.23109619146014
2900000 37.150988436168305
3000000 35

29500000 32.27907201144117
29600000 29.57353284571166
29700000 33.42842273546547
29800000 27.574849953974724
29900000 16.509393510037572
30000000 32.00310394839368
30100000 29.669815067399604
30200000 32.352137600671185
30300000 19.69495634731006
30400000 22.70056546406646
30500000 32.35906188958425
30600000 29.08549702167046
30700000 32.83225461713916
30800000 28.66853468252598
30900000 29.639442395811354
31000000 28.02481636148662
31100000 24.982123061900726
31200000 13.372926699530675
31300000 26.663928359738904
31400000 28.7032003489264
31500000 28.786523637381737
31600000 28.00908736582539
31700000 32.362314601834846
31800000 29.64875226408468
31900000 31.246519073730504
32000000 12.422285255944677
32100000 30.571263360497237
32200000 32.08986057587302
32300000 30.28151961780382
32400000 31.723218647924625
32500000 11.053592617869818
32600000 30.695196965998264
32700000 33.280585377454756
32800000 30.43393426614796
32900000 30.48149020074478
33000000 29.993219886763335
33100000 28

59200000 23.99409308248075
59300000 24.85516558706859
59400000 23.77878955447793
59500000 24.973726454056827
59600000 25.607004766267636
59700000 11.495769249357968
59800000 27.637106377262732
59900000 26.83055528005139
60000000 27.797316243426792
60100000 27.23322332766638
60200000 28.072537237386214
60300000 28.0698538731603
60400000 27.1413736414669
60500000 27.75279829468325
60600000 15.36005104930211
60700000 22.01455063388718
60800000 25.552970874736435
60900000 25.007621127718767
61000000 24.920769703340117
61100000 24.250514341529453
61200000 12.757724704587055
61300000 21.835140374887064
61400000 24.607234858715035
61500000 24.466567887608214
61600000 24.08916273751581
61700000 25.335374169260128
61800000 15.327615208360658
61900000 23.848835243256328
62000000 27.46978645009155
62100000 27.041292061057376
62200000 28.076068344840444
62300000 27.555694455773217
62400000 28.650091023915046
62500000 27.07714829047789
62600000 27.84118203848932
62700000 27.916921517818185
62800000

89000000 22.842229918013743
89100000 12.722547954546188
89200000 18.757377106325503
89300000 23.137623559470043
89400000 23.359018225430646
89500000 12.288245111576884
89600000 22.388573505108884
89700000 22.445495655486653
89800000 24.06891332673602
89900000 24.453836605398628
90000000 25.706083968199557
90100000 25.321422442997523
90200000 25.021714064814326
90300000 25.191310087625546
90400000 25.531676251021167
90500000 24.285704452077653
90600000 25.24191620278017
90700000 22.450684306186442
90800000 22.707645977044468
90900000 10.615186562679822
91000000 20.757355985263054
91100000 23.828084876191326
91200000 21.071864482227276
91300000 8.892867838312716
91400000 22.738893086871236
91500000 22.141342347544263
91600000 17.096475285128747
91700000 16.75191120371749
91800000 22.545266992292138
91900000 23.154570819464382
92000000 22.59306338186921
92100000 25.818655805597157
92200000 24.076699257806457
92300000 25.121890415960564
92400000 25.048769558529173
92500000 24.3215233018022

118000000 22.66501071386993
118100000 23.503549528683216
118200000 24.05157738086017
118300000 22.099582971636625
118400000 22.87349179890259
118500000 21.319888620500205
118600000 22.02757310163172
118700000 22.005860631382482
118800000 20.061447091405505
118900000 22.495753165518522
119000000 12.66739239359744
119100000 16.941736378114012
119200000 7.390323225618048
119300000 14.134999977737262
119400000 18.470169223342353
119500000 20.838292352065093
119600000 21.170170982664647
119700000 21.63686303068876
119800000 20.619389459823534
119900000 22.553011188454494
120000000 20.72918847632475
120100000 22.333475904164814
120200000 24.589220241296726
120300000 21.438660466773
120400000 23.71309368872832
120500000 20.664255893654037
120600000 20.52636523788444
120700000 20.684391681132222
120800000 19.64205236310079
120900000 20.862779915896425
121000000 20.075125086937735
121100000 19.143670111415897
121200000 7.92868996499268
121300000 12.844164916482162
121400000 14.426803539144093
1

...

290200000 4.590892669921579
290300000 5.1388950191483875
290400000 4.923955315156642
290500000 5.090282830639463
290600000 5.062698973455298
290700000 4.656144202936343
290800000 3.526492362189027
290900000 1.1013285976492067
291000000 1.1054814422621615
291100000 5.839816321263732
291200000 4.630619448238929
291300000 5.112699868830184
291400000 5.0330454467398384
291500000 4.884152097683626
291600000 4.795134261567022
291700000 4.747258824346539
291800000 5.204789925966589
291900000 4.844673104978854
292000000 5.981244904149436
292100000 5.253080669798096
292200000 5.084663120012002
292300000 5.0450586602951235
292400000 4.5848856163309994
292500000 5.047900597883129
292600000 4.8334978617705655
292700000 5.100156616495184
292800000 4.668455573234323
292900000 5.072451762039843
293000000 1.0844838893320798
293100000 1.0993786621378097
293200000 3.8874947623084837
293300000 4.601590443679297
293400000 4.788587216284673
293500000 4.953974881588429
293600000 4.806894735643031
293700000 

319200000 3.8279084147159588
319300000 3.38915895414688
319400000 3.6243870793963557
319500000 3.490401443407323
319600000 3.6821926810492847
319700000 3.6702301298997795
319800000 3.6401014827860245
319900000 4.397669166726504
320000000 4.676304772198331
320100000 4.037409789149444
320200000 4.040115749101343
320300000 3.332474446488726
320400000 4.215722918124003
320500000 4.183518911759342
320600000 3.4673413324157196
320700000 3.323177939851998
320800000 4.845547034778671
320900000 1.1553399301028133
321000000 0.6726600822756218
321100000 1.922540932222504
321200000 3.6645973376785994
321300000 3.0670910693902855
321400000 3.4913028906934573
321500000 3.7511563819922924
321600000 3.9488758286997308
321700000 3.9687417322480005
321800000 3.615203068033024
321900000 3.396616996893728
322000000 4.841050080487184
322100000 5.69983750091492
322200000 3.977559682293309
322300000 4.83621219011116
322400000 2.908065983738609
322500000 4.121272897183799
322600000 4.076747805070837
322700000

347900000 2.7269838760631577
348000000 1.7367474661131177
348100000 1.2987609842222612
348200000 1.22599161451872
348300000 1.2171237377934894
348400000 2.0774019594845328
348500000 1.6489610546894133
348600000 1.4011694289319778
348700000 1.31332567098179
348800000 0.5039893178710401
348900000 0.32786151620761156
349000000 0.8290843287033327
349100000 1.3449876404134629
349200000 1.2781314494761593
349300000 1.2817911113491238
349400000 1.3319156493687807
349500000 1.1629210189634984
349600000 1.744796649007465
349700000 1.3604659622926116
349800000 1.348195612804164
349900000 1.4092777849724645
350000000 1.5183961893363656
350100000 1.3187690802778242
350200000 1.3004875254878832
350300000 1.2720806592172345
350400000 2.1853730708326817
350500000 1.2516175460411536
350600000 1.2978548025981307
350700000 1.2768548866943281
350800000 1.4154836964379873
350900000 1.0293730201549651
351000000 0.337110911663411
351100000 0.3361075869746937
351200000 1.353970466918395
351300000 1.251415601

375900000 0.1659924017178085
376000000 0.1656885809886462
376100000 0.1655012209326267
376200000 0.1648284192760547
376300000 0.16878981588093572
376400000 0.1686875609660268
376500000 0.168281264218639
376600000 0.1682564226079398
376700000 0.16757934377741218
376800000 0.16680704824489953
376900000 0.16678930295855865
377000000 0.16631599748942255
377100000 0.16615414998711348
377200000 0.2211545288266679
377300000 0.17315821665849485
377400000 0.2788730725206084
377500000 0.2779840453097032
377600000 0.27769548004031974
377700000 0.17014008123112853
377800000 0.16326881820790567
best so far: 0
type: [1, 1, 3, 1, 43] 129
cases of this type: 30769209
377900000 0.9326124109304904
378000000 0.9551128796747778
378100000 0.9576939242529173
378200000 1.2821236835816523
378300000 1.090394290468216
378400000 1.261242079708832
378500000 0.912616742015014
378600000 1.12375540316694
378700000 0.8953332148747855
378800000 1.0887872297102272
378900000 1.189017550153764
379000000 0.970639639866999

404000000 0.8111820542219103
404100000 0.6443876770245969
404200000 0.7527790085712585
404300000 0.7238616436265687
404400000 0.7685503907372958
404500000 0.727456778082892
404600000 0.6435615198080679
404700000 0.8146003125401623
404800000 0.6541367125531634
404900000 0.709955087453568
405000000 0.7493428550312732
405100000 0.7461443519144196
405200000 0.7362864905911465
405300000 0.6299217557043798
405400000 0.7891629610920614
405500000 0.6784840509869317
405600000 0.6702408302020901
405700000 0.7634274433204677
405800000 0.7229620515130163
405900000 0.6995487724286152
406000000 0.6109305415375869
406100000 0.7325717282242318
406200000 0.6834388482545058
406300000 0.6512103133826064
406400000 0.7031197889258186
406500000 0.7168560044344445
406600000 0.6240255077587081
406700000 0.5921159935002566
406800000 0.6409638301977012
406900000 0.6865675035360204
407000000 0.6137571636715015
407100000 0.6068156937815103
407200000 0.7688498142013026
407300000 0.668443970808359
407400000 0.62589